# ArtifactGEN Exploration
Visualize real vs synthetic EEG windows; update paths as needed.

In [ ]:
import json, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
man = Path('../results/manifest.json')
items = json.loads(man.read_text()) if man.exists() else []
len(items)